In [17]:
#기본 모듈 불러오기
import os
import json
import pandas as pd
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

# 전처리
- json 형태 값 -> 데이터 프레임으로 변환
- 'date','visitStartTime': datetime으로 변경
    - 'visitStartTime' : POSIX time (협정 세계시 1970년 1월 1일 00:00:00부터 경과한 시간을 초로 환산)
- 범주형 변수 인코딩

In [2]:
# json 형태 -> 데이터 프레임으로 변환
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = pd.json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

df = load_df('C:/Users/82104/Desktop/스파르타_데이터부트캠프/VSCODE/과제2/train.csv')
df.head(1)

In [22]:
# date 열 int형 -> 날짜형으로 변경
df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'],unit='s')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype         
---  ------                                             --------------   -----         
 0   channelGrouping                                    903653 non-null  object        
 1   date                                               903653 non-null  datetime64[ns]
 2   fullVisitorId                                      903653 non-null  object        
 3   sessionId                                          903653 non-null  object        
 4   socialEngagementType                               903653 non-null  object        
 5   visitId                                            903653 non-null  int64         
 6   visitNumber                                        903653 non-null  int64         
 7   visitStartTime                                     903653 non-null  datetime64[ns]
 8   devi

In [ ]:
#각 열의 고유 값 확인하기
# group = df['trafficSource_adContent'].unique()

In [13]:
# 모든 컬럼 value_count 확인 & 저장하기
""" for column in df.columns:
    vc = df[column].value_counts()
    vc.to_csv(f"{column}_value_counts.csv") """

In [26]:
# 범주형 변수 -> 수치형 변수로 변환
# 인코딩 하는 이유: 범주형 변수를 넣는 경우 에러가 발생할 수 있어 수치형으로 변환하는 과정이 필요하다

# 라벨인코딩
label_encoder = LabelEncoder()
object_cols = df.select_dtypes(include=['object']).columns
for col in object_cols:
    df[col] = label_encoder.fit_transform(df[col])

df.head(3)


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
0,4,2016-09-02,80509,101785,0,1472830385,1,2016-09-02 15:33:05,11,0,...,0,1,1475,8,2,17774,2,1,44,1
1,4,2016-09-02,269007,340201,0,1472880147,1,2016-09-03 05:22:27,16,0,...,0,1,1475,8,2,17774,2,1,44,1
2,4,2016-09-02,277678,351204,0,1472865386,1,2016-09-03 01:16:26,11,0,...,0,1,1475,8,2,17774,2,1,44,1
